# Market Basket Analysis

The "market basket analysis" gives the opportunity to look into the carts of the consumers to find out they buy together also. It allows retailers to predict customer purchase tendencies and, hence, decide which products to cross-promote.


One of the techniques that is used for association rule mining (which is also called market basket analysis) is focusing on discovering the relations between different items in the dataset, as what products often are sold together.


Apriori Algorithm, basically, is the method among Market Basket Analysis as the name indicates. It operates on the hypothesis that, if most frequently bought items are those bought together means that any item subset out of that group is frequent as well. E.g., if the customers commonly purchase clusters of Grapes, Apple, and Mango, then they probably also make pairings of Grapes and Mango. The AI tool empowers retailers to research customers' purchasing trends and subsequently build on that data by changing store layouts, analysis of customer behavior, designing catalogs, or even making online suggestions of complementary products.


In [2]:
# %pip install mlxtend

  Obtaining dependency information for mlxtend from https://files.pythonhosted.org/packages/1c/07/512f6a780239ad6ce06ce2aa7b4067583f5ddcfc7703a964a082c706a070/mlxtend-0.23.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 1.2 MB/s eta 0:00:00a 0:00:010m
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

### Data

In [2]:
orders = pd.read_csv( './../data/orders.csv')
order_products_prior = pd.read_csv('./../data/order_products__prior.csv')
order_products_train = pd.read_csv( './../data/order_products__train.csv')
products = pd.read_csv('./../data/products.csv')
aisles = pd.read_csv('./../data/aisles.csv')
departments = pd.read_csv('./../data/departments.csv')

In [3]:
# Inspect the first few rows of each dataframe
print(order_products_prior.head())
print(order_products_train.head())
print(orders.head())
print(products.head())
print(aisles.head())
print(departments.head())

   order_id  product_id  add_to_cart_order  reordered
0         2       33120                  1          1
1         2       28985                  2          1
2         2        9327                  3          0
3         2       45918                  4          1
4         2       30035                  5          0
   order_id  product_id  add_to_cart_order  reordered
0         1       49302                  1          1
1         1       11109                  2          1
2         1       10246                  3          0
3         1       49683                  4          0
4         1       43633                  5          1
   order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2398795        1    prior             2          3                  7   
2    473747        1    prior             3          3                 12   
3   2254736        1    prior             4 

In [4]:
# Check for missing values
print(order_products_train.isnull().sum())
print(orders.isnull().sum())
print(order_products_prior.isnull().sum())
print(products.isnull().sum())
print(aisles.isnull().sum())
print(departments.isnull().sum())

order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64
order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64
order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64
product_id       0
product_name     0
aisle_id         0
department_id    0
dtype: int64
aisle_id    0
aisle       0
dtype: int64
department_id    0
department       0
dtype: int64


In [5]:
# Fill or drop missing values
orders['days_since_prior_order'].fillna(0)

0           0.0
1          15.0
2          21.0
3          29.0
4          28.0
           ... 
3421078    29.0
3421079    30.0
3421080    18.0
3421081     7.0
3421082    30.0
Name: days_since_prior_order, Length: 3421083, dtype: float64

In [6]:
#  Concatenate order_products_prior and order_products_train
order_products = pd.concat([order_products_prior, order_products_train])

# Check the shape of the concatenated DataFrame
print(order_products.shape)

(33819106, 4)


In [7]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [8]:
order_products.product_id.nunique()

49685

Out of 49685 keeping top 100 most frequent products.

In [9]:
product_counts = order_products.groupby('product_id')['order_id'].count().reset_index().rename(columns = {'order_id':'frequency'})
product_counts = product_counts.sort_values('frequency', ascending=False)[0:100].reset_index(drop = True)
product_counts = product_counts.merge(products, on = 'product_id', how = 'left')
product_counts.head(10)

,product_id,frequency,product_name,aisle_id,department_id
0,24852,491291,Banana,24,4
1,13176,394930,Bag of Organic Bananas,24,4
2,21137,275577,Organic Strawberries,24,4
3,21903,251705,Organic Baby Spinach,123,4
4,47209,220877,Organic Hass Avocado,24,4
5,47766,184224,Organic Avocado,24,4
6,47626,160792,Large Lemon,24,4
7,16797,149445,Strawberries,24,4
8,26209,146660,Limes,24,4
9,27845,142813,Organic Whole Milk,84,16


 100 most frequent items in order_products dataframe

In [10]:
freq_products = list(product_counts.product_id)
freq_products[1:10]

[13176, 21137, 21903, 47209, 47766, 47626, 16797, 26209, 27845]

In [11]:
len(freq_products)

100

In [12]:
order_products = order_products[order_products.product_id.isin(freq_products)]
order_products.shape

(7795471, 4)

In [13]:
order_products.order_id.nunique()

2444982

In [14]:
order_products = order_products.merge(products, on = 'product_id', how='left')
order_products.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2,28985,2,1,Michigan Organic Kale,83,4
1,2,17794,6,1,Carrots,83,4
2,3,24838,2,1,Unsweetened Almondmilk,91,16
3,3,21903,4,1,Organic Baby Spinach,123,4
4,3,46667,6,1,Organic Ginger Root,83,4


Structuring the data for feeding in the algorithm

In [15]:
basket = order_products.groupby(['order_id', 'product_name'])['reordered'].count().unstack().reset_index().fillna(0).set_index('order_id')
basket.head()

product_name,100% Raw Coconut Water,100% Whole Wheat Bread,2% Reduced Fat Milk,Apple Honeycrisp Organic,Asparagus,Bag of Organic Bananas,Banana,Bartlett Pears,Blueberries,Boneless Skinless Chicken Breasts,...,Sparkling Natural Mineral Water,Sparkling Water Grapefruit,Spring Water,Strawberries,Uncured Genoa Salami,Unsalted Butter,Unsweetened Almondmilk,Unsweetened Original Almond Breeze Almond Milk,Whole Milk,Yellow Onions
order_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
del product_counts, products, order_products, order_products_prior, order_products_train

encoding the units

In [17]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1 
    
basket = basket.applymap(encode_units)
basket.head()

product_name,100% Raw Coconut Water,100% Whole Wheat Bread,2% Reduced Fat Milk,Apple Honeycrisp Organic,Asparagus,Bag of Organic Bananas,Banana,Bartlett Pears,Blueberries,Boneless Skinless Chicken Breasts,...,Sparkling Natural Mineral Water,Sparkling Water Grapefruit,Spring Water,Strawberries,Uncured Genoa Salami,Unsalted Butter,Unsweetened Almondmilk,Unsweetened Original Almond Breeze Almond Milk,Whole Milk,Yellow Onions
order_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
basket.size

244498200

In [19]:
basket.shape

(2444982, 100)

Creating frequent sets and rules

In [20]:
frequent_items = apriori(basket, min_support=0.01, use_colnames=True, low_memory=True)
frequent_items.head()

/Users/aadityakasbekar/anaconda3/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.016062,(100% Raw Coconut Water)
1,0.025814,(100% Whole Wheat Bread)
2,0.015800,(2% Reduced Fat Milk)
3,0.035694,(Apple Honeycrisp Organic)
4,0.029101,(Asparagus)


In [21]:
frequent_items.tail()

,support,itemsets
124,0.010235,"(Organic Blueberries, Organic Strawberries)"
125,0.010966,"(Organic Hass Avocado, Organic Raspberries)"
126,0.017314,"(Organic Hass Avocado, Organic Strawberries)"
127,0.014533,"(Organic Raspberries, Organic Strawberries)"
128,0.010130,"(Organic Strawberries, Organic Whole Milk)"


In [22]:
frequent_items.shape

(129, 2)

In [23]:
rules = association_rules(frequent_items, metric="lift", min_threshold=1)
rules.sort_values('lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
35,(Limes),(Large Lemon),0.059984,0.065764,0.011860,0.197723,3.006544,0.007915,1.164480,0.709980
34,(Large Lemon),(Limes),0.065764,0.059984,0.011860,0.180345,3.006544,0.007915,1.146843,0.714372
52,(Organic Raspberries),(Organic Strawberries),0.058325,0.112711,0.014533,0.249174,2.210731,0.007959,1.181751,0.581582
53,(Organic Strawberries),(Organic Raspberries),0.112711,0.058325,0.014533,0.128940,2.210731,0.007959,1.081069,0.617230
37,(Organic Avocado),(Large Lemon),0.075348,0.065764,0.010538,0.139862,2.126728,0.005583,1.086147,0.572966
36,(Large Lemon),(Organic Avocado),0.065764,0.075348,0.010538,0.160244,2.126728,0.005583,1.101097,0.567088
47,(Organic Strawberries),(Organic Blueberries),0.112711,0.042956,0.010235,0.090809,2.114024,0.005394,1.052633,0.593909
46,(Organic Blueberries),(Organic Strawberries),0.042956,0.112711,0.010235,0.238274,2.114024,0.005394,1.164840,0.550621
49,(Organic Raspberries),(Organic Hass Avocado),0.058325,0.090339,0.010966,0.188018,2.081257,0.005697,1.120298,0.551699
48,(Organic Hass Avocado),(Organic Raspberries),0.090339,0.058325,0.010966,0.121389,2.081257,0.005697,1.071777,0.571115
